<a href="https://colab.research.google.com/github/knickka/Comment-Toxicity/blob/main/Comment_toxicity_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np


In [2]:
path = '/content/drive/MyDrive/Colab Notebooks/Toxic_Comment_Detection/toxci_comment_train.csv'
df = pd.read_csv(path)


In [3]:
df.iloc[43]['comment_text']

'FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!'

In [4]:
df[df.columns[2:]].iloc[43]

toxic            1
severe_toxic     0
obscene          1
threat           0
insult           1
identity_hate    0
Name: 43, dtype: int64

## Preprocess

In [5]:
#imports
from tensorflow.keras.layers import TextVectorization

In [6]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [7]:
MAX_WORDS = 200000 #number of words in our vocab


In [8]:
vectorizer = TextVectorization(max_tokens=MAX_WORDS,
                                  output_sequence_length = 1800,
                                  output_mode = 'int')

In [9]:
vectorizer.adapt(X.values)

In [10]:
vectorized_text = vectorizer(X.values)

In [11]:
len(X)

159571

In [12]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [13]:
def Save_Vectorizer(vectorizer,path):
  vectorizer_model = tf.keras.models.Sequential()
  vectorizer_model.add(tf.keras.Input(shape=(1,),dtype=tf.string))
  vectorizer_model.add(vectorizer)
  res = vectorizer_model.summary()

  vectorizer_model.save(path,save_format='tf')
  print(res)

In [14]:
path = '/content/drive/MyDrive/Colab Notebooks/Toxic_Comment_Detection/vectorizer'
Save_Vectorizer(vectorizer,path)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 1800)             0         
 torization)                                                     
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


None


In [15]:
#MCSHBP - map, cache, shuffle, batch prefetch from_tensor_slices, from_list

dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [16]:
dataset.as_numpy_iterator().next()

(array([[5229,    8,   25, ...,    0,    0,    0],
        [  40,   35, 1735, ...,    0,    0,    0],
        [  38,    9,   15, ...,    0,    0,    0],
        ...,
        [  46,  168,   23, ...,    0,    0,    0],
        [3720,   94,  123, ...,    0,    0,    0],
        [  27,    7, 1669, ...,    0,    0,    0]]),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]))

In [17]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [18]:
train_generator = train.as_numpy_iterator()

In [19]:
#imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Bidirectional, Embedding

In [20]:
model = Sequential()
model.add(Embedding(MAX_WORDS+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(6,activation='sigmoid'))


In [21]:
model.compile(loss='BinaryCrossentropy',optimizer='adam')

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                      

In [23]:
history = model.fit(train,epochs=15,validation_data=val)

Epoch 1/15
6981/6981 [==============================] - 779s 111ms/step - loss: 0.0639 - val_loss: 0.0485
Epoch 2/15
6981/6981 [==============================] - 626s 90ms/step - loss: 0.0465 - val_loss: 0.0406
Epoch 3/15
6981/6981 [==============================] - 631s 90ms/step - loss: 0.0412 - val_loss: 0.0358
Epoch 4/15
6981/6981 [==============================] - 627s 90ms/step - loss: 0.0368 - val_loss: 0.0332
Epoch 5/15
6981/6981 [==============================] - 618s 89ms/step - loss: 0.0332 - val_loss: 0.0286
Epoch 6/15
6981/6981 [==============================] - 625s 89ms/step - loss: 0.0299 - val_loss: 0.0284
Epoch 7/15
6981/6981 [==============================] - 620s 89ms/step - loss: 0.0269 - val_loss: 0.0236
Epoch 8/15
6981/6981 [==============================] - 620s 89ms/step - loss: 0.0242 - val_loss: 0.0204
Epoch 9/15
6981/6981 [==============================] - 617s 88ms/step - loss: 0.0222 - val_loss: 0.0182
Epoch 10/15
6981/6981 [==============================]

In [24]:
batch_x,batch_y = test.as_numpy_iterator().next()


In [25]:
(model.predict(batch_x)>0.5).astype(int)

1/1 [==============================] - 1s 724ms/step


array([[0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [26]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [27]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [28]:
from tensorflow.python.ops.array_ops import batch_gather
for batch in test.as_numpy_iterator():
  x_true,y_true = batch

  yhat = model.predict(x_true)

  y_true = y_true.flatten()
  yhat = yhat.flatten()

  pre.update_state(y_true,yhat)
  re.update_state(y_true,yhat)
  acc.update_state(y_true,yhat)

1/1 [==============================] - 0s 56ms/step


In [29]:
print(re.result().numpy())
print(pre.result().numpy())
print(acc.result().numpy())

0.9459534
0.9520223
0.4553661


In [30]:
path_m = '/content/drive/MyDrive/Colab Notebooks/Toxic_Comment_Detection/model'
model.save(path_m,save_format='tf')